# Spam Classification
### Used 2 Models Baseline Dense and LSTM

In [112]:
import numpy as np 
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Activation, Embedding, LSTM
import pandas as pd

In [102]:
data = pd.read_csv("../spam.csv", encoding='iso-8859-1')

In [103]:
print(data.columns.values)

['v1' 'v2' 'Unnamed: 2' 'Unnamed: 3' 'Unnamed: 4']


In [104]:
y_attribute_sub = dict(zip(['spam', 'ham'], [1, 0]))
for i in range(0, len(data)):
    data['v1'][i] = y_attribute_sub[data['v1'][i]]

In [105]:
print(data.head())

  v1                                                 v2 Unnamed: 2 Unnamed: 3  \
0  0  Go until jurong point, crazy.. Available only ...        NaN        NaN   
1  0                      Ok lar... Joking wif u oni...        NaN        NaN   
2  1  Free entry in 2 a wkly comp to win FA Cup fina...        NaN        NaN   
3  0  U dun say so early hor... U c already then say...        NaN        NaN   
4  0  Nah I don't think he goes to usf, he lives aro...        NaN        NaN   

  Unnamed: 4  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  


In [106]:
data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)
print(data.head())

  v1                                                 v2
0  0  Go until jurong point, crazy.. Available only ...
1  0                      Ok lar... Joking wif u oni...
2  1  Free entry in 2 a wkly comp to win FA Cup fina...
3  0  U dun say so early hor... U c already then say...
4  0  Nah I don't think he goes to usf, he lives aro...


Naive Method:

Using Dictionary

In [36]:
"""corpus = list()
for i in range(0, len(data)):
    corpus.extend(re.sub(r'[^\w]', ' ', data['v2'][i]).strip().split())
corpus = list(set(corpus))
print(len(corpus))"""

10864


In [37]:
"""encoding = dict(zip(corpus, [i+1 for i in range(0, len(corpus))]))"""

In [38]:
"""max_length = 0
for i in range(0, len(data)):
    vector = re.sub(r'[^\w]', ' ', data['v2'][i]).strip().split()
    for j in range(0, len(vector)):
        vector[j] = encoding[vector[j]]
    data['v2'][i] = vector
    if(max_length < len(vector)):
        max_length = len(vector)
print(data.head(), max_length)"""

  v1                                                 v2
0  0  [385, 5646, 5394, 759, 6982, 5545, 2620, 946, ...
1  0                 [2941, 638, 726, 5094, 3673, 3798]
2  1  [4376, 6104, 946, 6165, 6497, 2132, 5056, 4765...
3  0  [239, 2479, 6063, 566, 1061, 6776, 239, 3926, ...
4  0  [1862, 1930, 5116, 9961, 4940, 3371, 9909, 476... 190


In [39]:
"""print(data.head())
X = np.zeros((len(data['v2']), max_length))
for i in range(1, len(data)):
    data['v2'][i].extend([0 for j in range(0, max_length-len(data['v2'][i]))])
    X[i,:] = data['v2'][i]
print(data.head())"""

  v1                                                 v2
0  0  [385, 5646, 5394, 759, 6982, 5545, 2620, 946, ...
1  0                 [2941, 638, 726, 5094, 3673, 3798]
2  1  [4376, 6104, 946, 6165, 6497, 2132, 5056, 4765...
3  0  [239, 2479, 6063, 566, 1061, 6776, 239, 3926, ...
4  0  [1862, 1930, 5116, 9961, 4940, 3371, 9909, 476...
  v1                                                 v2
0  0  [385, 5646, 5394, 759, 6982, 5545, 2620, 946, ...
1  0  [2941, 638, 726, 5094, 3673, 3798, 0, 0, 0, 0,...
2  1  [4376, 6104, 946, 6165, 6497, 2132, 5056, 4765...
3  0  [239, 2479, 6063, 566, 1061, 6776, 239, 3926, ...
4  0  [1862, 1930, 5116, 9961, 4940, 3371, 9909, 476...


Using Tokenizer from keras with tf-idf

In [122]:
X = data.v2
y = data.v1
le_obj = LabelEncoder()
y = le_obj.fit_transform(y)
y = y.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [123]:
max_length = 150
max_words = 1000
token = Tokenizer(num_words = max_words)
token.fit_on_texts(X_train)
sequences = token.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences, maxlen = max_length)

In [114]:
def baseline_dense():
    model = Sequential()
    model.add(Dense(100, input_shape=(max_length, ), activation ='relu', name='dense_7'))
    model.add(Dense(30, activation='relu', name='dense_2'))
    model.add(Dense(1, activation='sigmoid', name='dense_3'))
    return model

In [115]:
def lstm():
    model = Sequential()
    model.add(Embedding(len(corpus), 16, input_length = max_length, name = 'embedding'))
    model.add(LSTM(64, return_sequences = True, name = 'lstm_1'))
    model.add(LSTM(32, return_sequences = False, name = 'lstm_2'))
    model.add(Dense(1, activation='sigmoid', name = 'dense_1'))
    return model

In [116]:
model = lstm()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 16)           173824    
_________________________________________________________________
lstm_1 (LSTM)                (None, 150, 64)           20736     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 207,009
Trainable params: 207,009
Non-trainable params: 0
_________________________________________________________________


In [117]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [118]:
print("Shape of input: {} and {} and shape of output: {}".format(X_train.shape, len(X_train[0]), y_train[0]))
model.fit(X_train, y_train, epochs=10, batch_size = 128)

Shape of input: (4457, 150) and 150 and shape of output: [0]
Epoch 1/10
4457/4457 [==============================] - 16s 4ms/step - loss: 0.4747 - acc: 0.8609
Epoch 2/10
4457/4457 [==============================] - 16s 4ms/step - loss: 0.2142 - acc: 0.9172
Epoch 3/10
4457/4457 [==============================] - 19s 4ms/step - loss: 0.0869 - acc: 0.9818
Epoch 4/10
4457/4457 [==============================] - 17s 4ms/step - loss: 0.0561 - acc: 0.9865
Epoch 5/10
4457/4457 [==============================] - 20s 4ms/step - loss: 0.0427 - acc: 0.9904
Epoch 6/10
4457/4457 [==============================] - 18s 4ms/step - loss: 0.0307 - acc: 0.9935
Epoch 7/10
4457/4457 [==============================] - 17s 4ms/step - loss: 0.0253 - acc: 0.9939
Epoch 8/10
4457/4457 [==============================] - 18s 4ms/step - loss: 0.0178 - acc: 0.9960
Epoch 9/10
4457/4457 [==============================] - 19s 4ms/step - loss: 0.0148 - acc: 0.9966
Epoch 10/10
4457/4457 [==============================] - 

In [124]:
sequences = token.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(sequences, maxlen = max_length)
model.evaluate(X_test, y_test)

1115/1115 [==============================] - 4s 4ms/step


[0.10696884384462317, 0.9739910313901345]

Model with Dense Network

In [125]:
model = baseline_dense()
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 100)               15100     
_________________________________________________________________
dense_2 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 18,161
Trainable params: 18,161
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, )